In [ ]:
# Importing required libraries
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
import numpy as np
from celluloid import Camera
from scipy.interpolate import make_interp_spline
import ffmpeg

In [ ]:
# Using colors
blue_color_for_customer_body = "#3377e4"
black_color_for_customer_edge = "black"
gray_color_for_semathores = "#444b61"
gray_color_for_intersection = "#b3b3b2"
green_color = "#00e600"
red_color = "#ff2525"
yellow_color = "yellow"
milk_color_for_external_area = "#fcfcf7"

In [ ]:
# Max time in seconds
MAX_TIME = 600
# Move count (frame count)
MOVE_COUNT = int(MAX_TIME * 10)

In [ ]:
def read_input_ouput_times_from_file(file_name):
    times_file = open(file_name, "r")
    list_of_customers_times = times_file.read().split("\n")
    list_of_customers_times.pop()
    customer_count = len(list_of_customers_times)
    customers_times = np.zeros((customer_count, 2), dtype = np.float64)

    cur_customer = 0
    for cur_times in list_of_customers_times:
        input_time, output_time = cur_times.split(" ")
        if (float(input_time) < 2.5):
            customers_times = customers_times[:-1][:]
            continue
        customers_times[cur_customer][0] = float(input_time)
        customers_times[cur_customer][1] = float(output_time)
        cur_customer += 1
    times_file.close()
    
    return customers_times

In [ ]:
# Calculate PI 1 customer's moving in time
def calculate_moving_for_pi_1(time_in, time_out):
    x = np.linspace(-10, 19, 30)

    time_for_move = 1
    time_for_move_before_intersection = 1
    time_for_move_after_intersection = 1
    x_moving = np.full(MOVE_COUNT, -10, dtype = np.float64)
    one_move_size = MAX_TIME / MOVE_COUNT
    
    i = 0
    start_pos = 0
    for cur_move in range(MOVE_COUNT):
        cur_time = one_move_size * cur_move
        if cur_time < time_in - time_for_move_before_intersection:
            continue
        elif cur_time < time_in:
            x_moving[cur_move] = x[i]
            i += 1
        elif cur_time < time_out - time_for_move:
            x_moving[cur_move] = start_pos
        elif cur_time < time_out + time_for_move_after_intersection:
            x_moving[cur_move] = x[i]
            i += 1
        else:
            break
    return x_moving

In [ ]:
# Calculate PI 2 trajectory
def pi_2_trajectory(vert_point_count = 1000, circle_point_count = 1000, hor_point_count = 1000):
    center_x = 6
    center_y = 5
    radius = 1

    x_vert = np.full(vert_point_count, 5, dtype = np.float64)
    y_vert = np.linspace(10, 5, vert_point_count, dtype = np.float64, endpoint = False)

    x_circle = np.linspace(5, 6, circle_point_count, dtype = np.float64, endpoint = False)
    y_circle = np.zeros(circle_point_count, dtype = np.float64)
    y_circle = np.multiply((x_circle - center_x), (x_circle - center_x))
    y_circle = radius * radius - y_circle
    y_circle = np.sqrt(y_circle)
    y_circle = center_y - y_circle

    x_hor = np.linspace(6, 10, hor_point_count, dtype = np.float64)
    y_hor = np.full(hor_point_count, 4, dtype = np.float64)
    x = np.concatenate((x_vert, x_circle, x_hor), axis = 0)
    y = np.concatenate((y_vert, y_circle, y_hor), axis = 0)
    
    return x, y

In [ ]:
# Calculate PI 2 customer's moving in time
def calculate_moving_for_pi_2(time_in, time_out):
    count_for_sec = MOVE_COUNT / MAX_TIME
    vert_point_count = int(count_for_sec // 3)
    circle_point_count = int(count_for_sec // 3 + count_for_sec % 3)
    hor_point_count = int(count_for_sec // 3)
    x_on_intersecection, y_on_intersecection = pi_2_trajectory(vert_point_count, circle_point_count, hor_point_count)

    x_before_intersection = np.full(10, 5)
    y_before_intersection = np.linspace(20, 10, 10, endpoint=False)

    x_after_intersection = np.linspace(11, 20, 10)
    y_after_intersection = np.full(10, 4)
    all_x = np.concatenate((x_before_intersection, x_on_intersecection, x_after_intersection), axis = 0)
    all_y = np.concatenate((y_before_intersection, y_on_intersecection, y_after_intersection), axis = 0)

    time_for_move = 1
    time_for_move_before_intersection = 1
    time_for_move_after_intersection = 1
    x_moving = np.full(MOVE_COUNT, -10, dtype = np.float64)
    y_moving = np.full(MOVE_COUNT, -10, dtype = np.float64)
    one_move_size = MAX_TIME / MOVE_COUNT
    
    i = 0
    start_pos_x = 5
    start_pos_y = 10
    for cur_move in range(MOVE_COUNT):
        cur_time = one_move_size * cur_move
        if cur_time < time_in - time_for_move_before_intersection:
            continue
        elif cur_time < time_in:
            x_moving[cur_move] = all_x[i]
            y_moving[cur_move] = all_y[i]
            i += 1
        elif cur_time < time_out - time_for_move:
            x_moving[cur_move] = start_pos_x
            y_moving[cur_move] = start_pos_y
        elif cur_time < time_out + time_for_move_after_intersection:
            x_moving[cur_move] = all_x[i]
            y_moving[cur_move] = all_y[i]
            i += 1
        else:
            break
    return x_moving, y_moving

In [ ]:
# Calculate PI 3 trajectory
def pi_3_trajectory(circle_point_count = 1000, hor_point_count = 1000):
    center_x = 8
    center_y = 6
    radius = 0.5

    x_circle = np.linspace(7.5, 8, circle_point_count, dtype = np.float64, endpoint = False)
    y_circle = np.zeros(circle_point_count, dtype = np.float64)
    y_circle = np.multiply((x_circle - center_x), (x_circle - center_x))
    y_circle = radius * radius - y_circle
    y_circle = np.sqrt(y_circle)
    y_circle = center_y + y_circle

    x_hor = np.linspace(8, 15, hor_point_count, dtype = np.float64)
    y_hor = np.full(hor_point_count, 6.5, dtype = np.float64)
    x = np.concatenate((x_circle, x_hor), axis = 0)
    y = np.concatenate((y_circle, y_hor), axis = 0)
    
    return x, y

In [ ]:
# Calculate PI 3 customer's moving in time
def calculate_moving_for_pi_3(time_in, time_out):
    circle_point_count = 25
    hor_point_count = 175

    x_before_on_intersection = np.full(120, 7.5)
    y_before_on_intersecection = np.linspace(1.2, 6, 120, endpoint = False)

    x_after_intersection, y_after_intersection = pi_3_trajectory(circle_point_count, hor_point_count)

    all_x = np.concatenate((x_before_on_intersection, x_after_intersection), axis = 0)
    all_y = np.concatenate((y_before_on_intersecection, y_after_intersection), axis = 0)

    time_for_move = 10
    time_for_move_before_intersection = 2
    time_for_move_after_intersection = 20
    x_moving = np.full(MOVE_COUNT, -10, dtype = np.float64)
    y_moving = np.full(MOVE_COUNT, -10, dtype = np.float64)
    one_move_size = MAX_TIME / MOVE_COUNT
    
    i = 0
    start_pos_x = 7.5
    start_pos_y = 2
    for cur_move in range(MOVE_COUNT):
        cur_time = one_move_size * cur_move
        if cur_time < time_in - time_for_move_before_intersection:
            continue
        elif cur_time < time_in:
            x_moving[cur_move] = all_x[i]
            y_moving[cur_move] = all_y[i]
            i += 1
        elif cur_time < time_out - time_for_move:
            x_moving[cur_move] = start_pos_x
            y_moving[cur_move] = start_pos_y
        elif cur_time < time_out + time_for_move_after_intersection:
            x_moving[cur_move] = all_x[i]
            y_moving[cur_move] = all_y[i]
            i += 1
        else:
            break
    return x_moving, y_moving

In [ ]:
# Calculate queue length in time
def calculate_queue_in_time(customers_times):
    queue_length_in_time = np.zeros(MOVE_COUNT, dtype = np.int64)
    scale = MOVE_COUNT / MAX_TIME
    for input_time, output_time in customers_times:
        input_move = int(input_time * scale)
        output_move = int(output_time * scale)
        queue_length_in_time[input_move:output_move] = np.add(queue_length_in_time[input_move:output_move], 1)

    return queue_length_in_time

In [ ]:
# Read data
pi_1_customers_times = read_input_ouput_times_from_file(file_name = "pi_1_times_io.txt")
pi_2_customers_times = read_input_ouput_times_from_file(file_name = "pi_2_times_io.txt")
pi_3_customers_times = read_input_ouput_times_from_file(file_name = "pi_3_times_io.txt")

# Calculate moving for all customers
all_pi_1_customers_moving = []
for input_time, output_time in pi_1_customers_times:
    all_pi_1_customers_moving.append(calculate_moving_for_pi_1(input_time, output_time))

all_pi_2_customers_x_moving = []
all_pi_2_customers_y_moving = []
for input_time, output_time in pi_2_customers_times:
    x, y = calculate_moving_for_pi_2(input_time, output_time)
    all_pi_2_customers_x_moving.append(x)
    all_pi_2_customers_y_moving.append(y)

all_pi_3_customers_x_moving = []
all_pi_3_customers_y_moving = []
for input_time, output_time in pi_3_customers_times:
    x, y = calculate_moving_for_pi_3(input_time, output_time)
    all_pi_3_customers_x_moving.append(x)
    all_pi_3_customers_y_moving.append(y)

# Calculate queue length
pi_1_queue_length_in_time = calculate_queue_in_time(pi_1_customers_times)
pi_2_queue_length_in_time = calculate_queue_in_time(pi_2_customers_times)
pi_3_queue_length_in_time = calculate_queue_in_time(pi_3_customers_times)

In [ ]:
def read_greem_light_time_from_file(file_name):
    times_file = open(file_name, "r")
    list_of_greenlight_times = times_file.read().split("\n")
    list_of_greenlight_times.pop()
    str_count = len(list_of_greenlight_times)
    greenlights_times = np.zeros((str_count, 2), dtype = np.float64)

    cur_str = 0
    for cur_times in list_of_greenlight_times:
        start_time, end_time = cur_times.split(" ")
        greenlights_times[cur_str][0] = float(start_time)
        greenlights_times[cur_str][1] = float(end_time)
        cur_str += 1
    times_file.close()
    
    return greenlights_times

In [ ]:
# Read green light time data
pi_1_greem_light_time = read_greem_light_time_from_file(file_name = "pi_1_times_greenlight.txt")
pi_2_greem_light_time = read_greem_light_time_from_file(file_name = "pi_2_times_greenlight.txt")
pi_3_greem_light_time = read_greem_light_time_from_file(file_name = "pi_3_times_greenlight.txt")

In [ ]:
def draw_framing_frame(ax):
    ax.add_patch(
        patches.FancyBboxPatch(
            (-5, 1),
            20,
            14,
            edgecolor = 'white',
            fill=False,
            zorder=5,
            boxstyle="round,rounding_size=0.3",
            linewidth=15
        ) )

    ax.add_patch(
        patches.FancyBboxPatch(
            (-4.61, 1.39),
            19.22,
            13.22,
            edgecolor = 'black',
            fill=False,
            zorder=5,
            boxstyle="round,rounding_size=0.3",
            linewidth=3
        ) )

In [ ]:
def draw_intersection_diagram(ax):
    #ax.hlines(2, -5, 15, linewidth=2, zorder=3, color = gray_color)
    ax.add_patch(
        patches.FancyBboxPatch(
        (-6, 0),
        22,
        2,
        edgecolor = gray_color_for_intersection,
        facecolor=milk_color_for_external_area,
        zorder=3,
        boxstyle="round,pad=-0.0004,rounding_size=0.3",
        linewidth=2
        ) )

    ax.add_patch(
        patches.FancyBboxPatch(
            (6.99, 6),
            10,
            10,
            edgecolor = gray_color_for_intersection,
            facecolor=milk_color_for_external_area,
            zorder=3,
            boxstyle="round,pad=-0.0004,rounding_size=0.3",
            linewidth=2
        ) )

    ax.add_patch(
        patches.FancyBboxPatch(
            (-7, 6),
            10,
            10,
            edgecolor = gray_color_for_intersection,
            facecolor=milk_color_for_external_area,
            zorder=3,
            boxstyle="round,pad=-0.0004,rounding_size=0.3",
            linewidth=2
        ) )

In [ ]:
def draw_intersection_borders(ax):
    ax.hlines(10.01, 3, 7, color = gray_color_for_intersection, zorder=2, lw = 1.5)
    ax.vlines(0, 2, 6, color = gray_color_for_intersection, zorder=2, lw = 1.5)
    ax.vlines(10, 2, 6, color = gray_color_for_intersection, zorder=2, lw = 1.5)

In [ ]:
def draw_flow_moving_trajectories(ax):
    line_style = "dotted"
    # Add a plot of PI_1 moving trajectory
    ax.hlines(4, 7.2, 0, lw = 1, zorder=1, linestyle=line_style, color = gray_color_for_intersection)
    ax.hlines(4, 8.8, 10, lw = 1, zorder=1, linestyle=line_style, color = gray_color_for_intersection)
    # Add a plot of PI_2 moving trajectory
    x, y = pi_2_trajectory()
    ax.plot(x[:1750], y[:1750], lw = 1, zorder=1, linestyle=line_style, color = gray_color_for_intersection)
    # Add a plot of PI_3 moving trajectory
    for i in range(10):
        ax.hlines(2.0 + i * 0.4, 7.3, 8.7, lw = 2, zorder=1, color = gray_color_for_intersection)

In [ ]:
def draw_semathore_borders(ax):
    ax.scatter(1, 5, color = gray_color_for_semathores, s = 250)
    ax.scatter(4, 9, color = gray_color_for_semathores, s = 250)
    ax.scatter(6.5, 3, color = gray_color_for_semathores, s = 250)

In [ ]:
def flow_customers_init(ax):
    pi_1_cutomers = ax.scatter(0, 0, zorder = 4, color = blue_color_for_customer_body, edgecolors = black_color_for_customer_edge, linewidths = 1.0, s = 200)
    pi_2_cutomers = ax.scatter(0, 0, zorder = 4, color = blue_color_for_customer_body, edgecolors = black_color_for_customer_edge, linewidths = 1.0, s = 200)
    pi_3_cutomers = ax.scatter(0, 0, zorder = 4, color = blue_color_for_customer_body, edgecolors = black_color_for_customer_edge, linewidths = 0.75, s = 10)

    return pi_1_cutomers, pi_2_cutomers, pi_3_cutomers

In [ ]:
def flow_queue_lengths_init(ax):
    pi_1_queue_len = ax.text(0.21, 0.265, '', transform=ax.transAxes, fontsize=15)
    pi_2_queue_len = ax.text(0.44, 0.66, '', transform=ax.transAxes, fontsize=15)
    pi_3_queue_len = ax.text(0.56, 0.02, '', transform=ax.transAxes, fontsize=15)

    return pi_1_queue_len, pi_2_queue_len, pi_3_queue_len

In [ ]:
def flow_greenlights_init(ax):
    pi_1_greenlight = ax.scatter(1, 5, color = red_color , s = 100)
    pi_2_greenlight = ax.scatter(4, 9, color = red_color , s = 100)
    pi_3_greenlight = ax.scatter(6.5, 3, color = red_color , s = 100)

    return pi_1_greenlight, pi_2_greenlight, pi_3_greenlight

In [ ]:
def change_greenlight_color(cur_time, greem_light_start, greem_light_end):
    do_we_need_to_change_color = True
    color = yellow_color
    increase_str_id_value = 0

    if (cur_time < greem_light_start - 3.0):
        do_we_need_to_change_color = False
    elif (cur_time < greem_light_start):
        color = yellow_color
    elif (cur_time < greem_light_end):
        color = green_color
    elif (cur_time < greem_light_end + 3):
        color = yellow_color
    else:
        color = red_color
        increase_str_id_value = 1
            
    return do_we_need_to_change_color, color, increase_str_id_value

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim([-5, 15])
ax.set_ylim([1, 15])
plt.style.use('seaborn-v0_8-white')

ax.axis('off')
draw_framing_frame(ax)
draw_intersection_diagram(ax)
draw_intersection_borders(ax)
draw_flow_moving_trajectories(ax)
draw_semathore_borders(ax)

pi_1_cutomers, pi_2_cutomers, pi_3_cutomers = flow_customers_init(ax)
pi_1_queue_len, pi_2_queue_len, pi_3_queue_len = flow_queue_lengths_init(ax)
pi_1_greenlight, pi_2_greenlight, pi_3_greenlight = flow_greenlights_init(ax)

greenlight_pi_1_str_id = 0
greenlight_pi_2_str_id = 0
greenlight_pi_3_str_id = 0
greenlight_pi_1_str_max_id = len(pi_1_greem_light_time)
greenlight_pi_2_str_max_id = len(pi_2_greem_light_time)
greenlight_pi_3_str_max_id = len(pi_3_greem_light_time)

In [ ]:
def animate(i):
    global greenlight_pi_1_str_id
    global greenlight_pi_2_str_id
    global greenlight_pi_3_str_id

    pi_1_points = np.array([ [all_pi_1_customers_moving[j][i], 4] for j in range(len(all_pi_1_customers_moving)) ] )
    pi_2_points = np.array([ [all_pi_2_customers_x_moving[j][i], all_pi_2_customers_y_moving[j][i]] for j in range(len(all_pi_2_customers_x_moving)) ] )
    pi_3_points = np.array([ [all_pi_3_customers_x_moving[j][i] + (j % 5) / 4, all_pi_3_customers_y_moving[j][i]] for j in range(len(all_pi_3_customers_x_moving)) ] )

    pi_1_cutomers.set_offsets(pi_1_points)
    pi_2_cutomers.set_offsets(pi_2_points)
    pi_3_cutomers.set_offsets(pi_3_points)
    if pi_1_queue_length_in_time[i] < 10:
        pi_1_queue_len.set_x(0.21)
    else:
        pi_1_queue_len.set_x(0.2)
    if pi_2_queue_length_in_time[i] < 10:
        pi_2_queue_len.set_x(0.437)
    else:
        pi_2_queue_len.set_x(0.428)
    if pi_3_queue_length_in_time[i] < 10:
        pi_3_queue_len.set_x(0.563)
    else:
        pi_3_queue_len.set_x(0.55)

    time_template = '%d'
    pi_1_queue_len.set_text(time_template % (pi_1_queue_length_in_time[i]))
    pi_2_queue_len.set_text(time_template % (pi_2_queue_length_in_time[i]))
    pi_3_queue_len.set_text(time_template % (pi_3_queue_length_in_time[i]))

    one_move_size = MAX_TIME / MOVE_COUNT
    cur_time = i * one_move_size

    if (greenlight_pi_1_str_id < greenlight_pi_1_str_max_id):
        do_we_need_to_change_color, color, increase_value = (
            change_greenlight_color(cur_time, pi_1_greem_light_time[greenlight_pi_1_str_id][0], pi_1_greem_light_time[greenlight_pi_1_str_id][1])
        )
        if (do_we_need_to_change_color):
            pi_1_greenlight.set_color(color)
        greenlight_pi_1_str_id += increase_value

    if (greenlight_pi_2_str_id < greenlight_pi_2_str_max_id):
        do_we_need_to_change_color, color, increase_value = (
            change_greenlight_color(cur_time, pi_2_greem_light_time[greenlight_pi_2_str_id][0], pi_2_greem_light_time[greenlight_pi_2_str_id][1])
        )
        if (do_we_need_to_change_color):
            pi_2_greenlight.set_color(color)
        greenlight_pi_2_str_id += increase_value

    if (greenlight_pi_3_str_id < greenlight_pi_3_str_max_id):
        do_we_need_to_change_color, color, increase_value = (
            change_greenlight_color(cur_time, pi_3_greem_light_time[greenlight_pi_3_str_id][0], pi_3_greem_light_time[greenlight_pi_3_str_id][1])
        )
        if (do_we_need_to_change_color):
            pi_3_greenlight.set_color(color)
        greenlight_pi_3_str_id += increase_value

    return pi_1_cutomers, pi_2_cutomers, pi_3_cutomers, pi_1_queue_len, pi_2_queue_len, pi_3_queue_len, pi_1_greenlight, pi_2_greenlight, pi_3_greenlight

In [ ]:
ani = FuncAnimation(fig, animate, repeat = True, frames = MOVE_COUNT, interval = 25, blit = False)
ani.save("mymovie.mp4", dpi = 300)